In [2]:
from nga.utils import load_config, plane_eval_ray_bundle, save_as_image, convert_to_transformed_space
from nerfstudio.utils.eval_utils import eval_load_checkpoint
from nerfstudio.cameras.rays import RayBundle
from nerfstudio.utils import colormaps
from nerfstudio.utils.io import load_from_json

from PIL import Image
from pathlib import Path
import yaml
import torch
import json
import numpy as np

In [3]:

def sphere_eval_ray_bundle(dataparser_transforms_data, near_z, n = 1001):
    dataparser_scale = dataparser_transforms_data["scale"]
    x = torch.linspace(-0.5, 0.5, n)
    y = torch.linspace(-0.5, 0.5, n)
    z = near_z
    grid_x, grid_y = torch.meshgrid(x, y)
    origins = torch.stack([grid_x, grid_y, z * torch.ones([n, n])], dim=-1)
    origins = convert_to_transformed_space(origins, dataparser_transforms_data)
    directions = torch.zeros_like(origins)
    directions[:, :, 2] = -1.0
    pixel_area = (dataparser_scale ** 2) * torch.ones((n, n, 1)) / (n ** 2)
    nears = torch.zeros((n, n, 1))
    fars = torch.ones((n, n, 1)) * 2 * near_z * dataparser_scale
    camera_indices = torch.zeros((n, n, 1))

    ray_bundle = RayBundle(
        origins=origins, directions=directions, pixel_area=pixel_area,
        camera_indices=camera_indices,
        nears=nears,
        # fars=fars, 
    )
    return ray_bundle

def load_metadata(config):
    if config.data.suffix == ".json":
        meta = load_from_json(config.data)
        data_dir = config.data.parent
    else:
        meta = load_from_json(config.data / "transforms.json")
        data_dir = config.data
    return meta, data_dir

In [4]:
# config_path = "outputs/pattern_plane1_720x480/tensorf/2023-09-01_085608/config.yml"
# config_path = "outputs/pattern_plane1_720x480/instant-ngp/2023-08-31_194733/config.yml"
# config_path = "outputs/pattern_plane1_720x480/nerfacto/2023-09-01_092209/config.yml"
# config_path = "outputs/pattern_plane1_720x480/nerfacto/2023-09-01_102747/config.yml"
# config_path = "outputs/pattern_plane1_720x480/nerfacto/2023-09-14_142127/config.yml"
config_path = "outputs/checkered_square_pyramid/nerfacto/2023-09-15_152357/config.yml"


config = load_config(config_path)
config.load_dir = config.get_checkpoint_dir()

meta, data_dir = load_metadata(config)


In [5]:

# setup pipeline (which includes the DataManager)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline = config.pipeline.setup(device=device, test_mode="val")
pipeline.eval()


checkpoint_path, step = eval_load_checkpoint(config, pipeline)
results_path = config.get_base_dir() / "results.json"
render_output_path = config.get_base_dir() / "renders"
render_output_path.mkdir(parents=True, exist_ok=True)

[15:25:35] Auto image downscale factor of 1                                                 ]8;id=461465;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=351359;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#349\349]8;;\

            Dataset is overriding train_indices to [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, ]8;id=355264;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=460077;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#200\200]8;;\
           20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,                              
           40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,                              
           60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79,                              
           80, 81, 82, 83, 84, 85, 86, 87, 88, 89]                                                                      

            Dataset is overriding val_indices to [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]   ]8;id=297564;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=537642;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#200\200]8;;\

Setting up training dataset...

Caching all 80 images.

...

Setting up evaluation dataset...

Caching all 10 images.

Output()

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/checkered_square_pyramid/nerfacto/2023-09-15_152357/nerfstudio_models/step-000004000.ckpt

In [6]:
import cv2
import OpenEXR, Imath

depth_filenames = pipeline.datamanager.eval_dataset.metadata["depth_filenames"]

def read_depth_map(file_path):
    exr_file = OpenEXR.InputFile(file_path)
    dw = exr_file.header()['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1
    
    pt = Imath.PixelType(Imath.PixelType.FLOAT)
    depth_str = exr_file.channel('R', pt)
    depth = np.frombuffer(depth_str, dtype=np.float32)
    depth.shape = (height, width)  # reshape
    
    return torch.unsqueeze(torch.from_numpy(depth), dim=-1).to(device=device)

for camera_ray_bundle, batch in pipeline.datamanager.fixed_indices_eval_dataloader:
    image_idx = batch["image_idx"]
    depth_filepath = depth_filenames[image_idx]
    depth = read_depth_map(str(depth_filepath))
    mask = depth <= 1000
    depth[depth > 1000] = 0
    print(torch.max(depth[mask]))
    outputs = pipeline.model.get_outputs_for_camera_ray_bundle(camera_ray_bundle)

    rgb = outputs["rgb"]
    acc = colormaps.apply_colormap(outputs["accumulation"])
    depth_pred = colormaps.apply_depth_colormap(
        outputs["depth"],
        accumulation=outputs["accumulation"],
    )
    depth_gt = colormaps.apply_depth_colormap(
        depth,
    )
    depth_gt = torch.concat([depth_gt, mask], dim=-1)

    save_as_image(rgb, render_output_path / f"rgb_pred_{image_idx:04d}.png")
    save_as_image(batch["image"], render_output_path / f"rgb_gt_{image_idx:04d}.png")
    save_as_image(acc, render_output_path / f"acc_{image_idx:04d}.png")
    save_as_image(depth_pred, render_output_path / f"depth_pred_{image_idx:04d}.png")
    save_as_image(depth_gt, render_output_path / f"depth_gt_{image_idx:04d}.png")
    
    

/tmp/ipykernel_60679/2119514443.py:17: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343998658/work/torch/csrc/utils/tensor_numpy.cpp:206.)
  return torch.unsqueeze(torch.from_numpy(depth), dim=-1).to(device=device)


tensor(4.5547, device='cuda:0')
tensor(4.4023, device='cuda:0')
tensor(4.1914, device='cuda:0')
tensor(4.1914, device='cuda:0')
tensor(4.4023, device='cuda:0')
tensor(4.5547, device='cuda:0')
tensor(4.4023, device='cuda:0')
tensor(4.1914, device='cuda:0')
tensor(4.1914, device='cuda:0')
tensor(4.4023, device='cuda:0')


In [66]:


# load data for coverting between original and normalized coodinate space
dataparser_transforms_path = config.get_base_dir() / "dataparser_transforms.json"
dataparser_transforms_data = json.load(open(dataparser_transforms_path))
dataparser_scale = dataparser_transforms_data["scale"]
transform = torch.tensor(dataparser_transforms_data["transform"])

near_z = 0.15
camera_ray_bundle = plane_eval_ray_bundle(dataparser_transforms_data, near_z).to(device)
outputs = pipeline.model.get_outputs_for_camera_ray_bundle(camera_ray_bundle)

rgb = outputs["rgb"]
acc = colormaps.apply_colormap(outputs["accumulation"])
depth = colormaps.apply_depth_colormap(
    outputs["depth"],
    accumulation=outputs["accumulation"],
)

z = near_z - (outputs["depth"] / dataparser_scale)
z_vis = colormaps.apply_depth_colormap(
    torch.clamp(z, -0.3, 0.3),
    accumulation=outputs["accumulation"],
)

save_as_image(rgb, render_output_path / "rgb.jpg")
save_as_image(acc, render_output_path / "acc.jpg")
save_as_image(depth, render_output_path / "depth.jpg")
save_as_image(z_vis, render_output_path / "z.jpg")
torch.save(z_vis, render_output_path / "z.pt")

# Get the output and define the names to save to
benchmark_info = {
    "experiment_name": config.experiment_name,
    "method_name": config.method_name,
    "checkpoint": str(checkpoint_path),
    "results": {
        "max_z": float(torch.max(z)),
        "min_z": float(torch.min(z)),
        "std_z": float(torch.std(z)),
        "mean_z": float(torch.mean(z)),
    },
}
# Save output to output file
results_path.write_text(json.dumps(benchmark_info, indent=2), "utf8")

353